In [1]:
import random

In [2]:
def dominados(poblacion, objetivos):
    frente = [[]]
    S = {tuple(p): [] for p in poblacion}
    n = {tuple(p): 0 for p in poblacion}
    rank = {}
    for p in poblacion:
        for q in poblacion:
            p_key, q_key = tuple(p), tuple(q)
            if all(obj(p) <= obj(q) for obj in objetivos) and any(obj(p) < obj(q) for obj in objetivos):
                S[p_key].append(q)
            elif all(obj(q) <= obj(p) for obj in objetivos) and any(obj(q) < obj(p) for obj in objetivos):
                n[p_key] += 1
        if n[p_key] == 0:
            rank[p_key] = 0
            frente[0].append(p)
    i = 0
    while frente[i]:
        next_front = []
        for p in frente[i]:
            for q in S[tuple(p)]:
                n[tuple(q)] -= 1
                if n[tuple(q)] == 0:
                    rank[tuple(q)] = i + 1
                    next_front.append(q)
        i += 1
        frente.append(next_front)
    return frente[:-1]

In [3]:
def crowd(front, objetivos):
    distancia = {tuple(p): 0.0 for p in front}
    if not front:
        return distancia
    for obj in objetivos:
        front_sorted = sorted(front, key=obj)
        min_val = obj(front_sorted[0])
        max_val = obj(front_sorted[-1])
        distancia[tuple(front_sorted[0])] = float('inf')
        distancia[tuple(front_sorted[-1])] = float('inf')
        for i in range(1, len(front_sorted) - 1):
            prev_val = obj(front_sorted[i - 1])
            next_val = obj(front_sorted[i + 1])
            if max_val > min_val:
                distancia[tuple(front_sorted[i])] += (next_val - prev_val) / (max_val - min_val)
    return distancia

In [4]:
def nsga2(objetivos, bounds, pop_size=50, generaciones=100,
          cruza_prob=0.9, muta_prob=0.1):
    pop = [[random.uniform(lb, ub) for lb, ub in bounds] for _ in range(pop_size)]

    for gen in range(generaciones):
        
        hijos = []
        while len(hijos) < pop_size:
            a, b = random.sample(pop, 2)
           
            if random.random() < cruza_prob:
                child = [(x + y) / 2 for x, y in zip(a, b)]
            else:
                child = a[:]
           
            for i, (lb, ub) in enumerate(bounds):
                if random.random() < muta_prob:
                    child[i] += random.gauss(0, 1) * (ub - lb) * 0.1
                    child[i] = min(max(child[i], lb), ub)
            hijos.append(child)

        union = pop + hijos
        frente = dominados(union, objetivos)

        new_pop = []
        for front in frente:
            if len(new_pop) + len(front) > pop_size:
                dist = crowd(front, objetivos)
                front_sorted = sorted(front, key=lambda p: dist[tuple(p)], reverse=True)
                new_pop.extend(front_sorted[:pop_size - len(new_pop)])
                break
            new_pop.extend(front)
        pop = new_pop

    pareto = dominados(pop, objetivos)[0]
    return pareto

In [5]:
if __name__ == "__main__":
    objetivos = [
        lambda x: x[0]**2 + x[1]**2,
        lambda x: (x[0] - 2)**2 + (x[1] + 1)**2,
    ]
    bounds = [(-5, 5), (-5, 5)]

    pareto_front = nsga2(objetivos, bounds,
                        pop_size=100,
                        generaciones=200,
                        cruza_prob=0.9,
                        muta_prob=0.1)

    print("Frente de Pareto (aproximado):")
    for sol in pareto_front:
        print(sol)


Frente de Pareto (aproximado):
[0.00030688785698108716, -0.0007085610738965381]
[1.9326810098900213, -1.001335403944967]
[0.00030688785698108716, -0.0007085610738965381]
[0.00030688785698108716, -0.0007085610738965381]
[1.9393982979950315, -0.8544325556570231]
[1.3690628235821718, -0.6714916996749246]
[1.3690628235821718, -0.6714916996749246]
[0.6948231423092462, -0.33917799224999023]
[0.7450884584238371, -0.36736971921413153]
[1.6592036191671968, -0.8144142434109392]
[0.21057581544719978, -0.10037847691509408]
[0.21057581544719978, -0.10037847691509408]
[1.8119124590888787, -0.9154128409843894]
[1.8119124590888787, -0.9154128409843894]
[1.8119124590888787, -0.9154128409843894]
[1.8119124590888787, -0.9154128409843894]
[1.8119124590888787, -0.9154128409843894]
[1.8119124590888787, -0.9154128409843894]
[0.4696315535427063, -0.23407354288791768]
[0.4696315535427063, -0.23407354288791768]
[1.4260873225185564, -0.6867367710067661]
[1.8446430410222445, -0.927883979800995]
[0.109527864375872